## Лабораторная работа №3. Коррекция опечаток
#### Предмет: Методы интеллектуального анализа текстов
#### Студент: Леонова Алина, СБ № 1032212306, НФИмд-01-21

### 3.4.	Написать функцию сравнения векторов, составленных из частотностей входящих в слово букв (CountVectorizer). Помимо верных вариантов слов из примеров использовать в небольшом словаре своего метода дополнительно похожие и непохожие на верные слова русского языка.


In [1]:
examples = ["вот в инете откапал такую инеерсную статейку предлагаю вашему внимани",
            "может и в правду лутше тебе молчать чем пытаться сказать",
            "утром мы сидели как сычи а потом каааак начали работать"]

In [2]:
# частотный словарь для русского языка
! wget -q https://github.com/Baksalyar/mc.hertzbeat.ru-Frequency-Dictionaries/raw/master/mc.hertzbeat.ru_frequency_dict.txt

In [3]:
# правильные слова для данного примера и другие существующие слова, близкие к ним
w = ['вот','в','инете','откопал','такую','интересную','статейку','предлагаю','вашему','вниманию','может','и','вправду','лучше','тебе','молчать','чем','пытаться','сказать','утром','мы','сидели','как','сычи','а','потом','как','начали','работать']
other_w = ['копал','капал','такая','интересно','возлогаю','внимание','тобой','молчишь','правда','сказка','после','потел','потемки']

freq_dict = []
with open("mc.hertzbeat.ru_frequency_dict.txt", "r") as f:
    for string in f:
        freq_dict.append(string.split()[0])   # только сами слова, без частот

# объединение нужных для примера слов, других существующих слов, близких к ним, и 1000 самых частотных слов
n_freq = 1000   # в словаре самых частотных слов 480092 слова, можно увеличить их количество, что, возможно, улучшит качество работы и уменьшит скорость
w_dict = w + other_w + freq_dict[:n_freq]

len(w_dict)   # размер итогового списка эталонных слов для сравнения

1042

In [4]:
from nltk.tokenize import RegexpTokenizer
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
import numpy as np
from numpy.linalg import norm
# from sklearn.metrics.pairwise import cosine_similarity,cosine_distances


def check_vectors(tokens, dict_w):
  # нахождение букв и биграм проверяемого слова
  # нахождение косинусной близости проверяемого слова со словами из словаря
  for t in tokens:
    best_sim = 0
    for w in dict_w:
      cv =  CountVectorizer(analyzer="char", ngram_range=(1,2))
      count_matrix = cv.fit_transform([t, w])
      count_array = count_matrix.toarray()
      df = pd.DataFrame(data=count_array,columns = cv.get_feature_names_out())
      #print(df)     # печать векторов для пары сравниваемых слов
      
      v_t = np.array(df.loc[0])
      v_w = np.array(df.loc[1])
      
      # косинусная близость
      cos_sim = round( np.dot(v_t, v_w)/(norm(v_t)*norm(v_w)) , 6)    # результаты совпадают с работой библиотечной
      #cos_sim = round(cosine_similarity(v_t.reshape(1,-1), v_w.reshape(1,-1))[0][0], 6)


      if cos_sim == 1:
        best_sim = cos_sim
        print(t, w, best_sim)
        break
      elif cos_sim > best_sim:   # запоминание наибольшей близости и слова
        best_sim = cos_sim
        best_word = w

    if best_sim != 1:
      print('* ', t, best_word, best_sim)


# тестирование
for row in examples:
  tokenizer=RegexpTokenizer('\W+', gaps=True)     
  t = tokenizer.tokenize(row)
  check_vectors(t, w_dict)
  print()

вот вот 1.0
в в 1.0
инете инете 1.0
*  откапал капал 0.856349
такую такую 1.0
*  инеерсную интересную 0.819028
статейку статейку 1.0
предлагаю предлагаю 1.0
вашему вашему 1.0
*  внимани вниманию 0.95119

может может 1.0
и и 1.0
в в 1.0
*  правду вправду 0.934199
*  лутше лучше 0.666667
тебе тебе 1.0
молчать молчать 1.0
чем чем 1.0
пытаться пытаться 1.0
сказать сказать 1.0

утром утром 1.0
мы мы 1.0
сидели сидели 1.0
как как 1.0
сычи сычи 1.0
а а 1.0
потом потом 1.0
*  каааак а 0.718421
начали начали 1.0
работать работать 1.0



Успешность работы зависит от словаря. Для ошибочного слова 'откапал' было добавлено правильное по контексту 'откопал' и ошибочное тут 'капал', но векторно второе оказалось ближе к первоначальному.

А также метод не поможет с большым количеством лишних букв, наиближайшим словом к 'каааак' оказалось 'а'.